<a href="https://colab.research.google.com/github/rickvelloso/Learning-IA/blob/main/EngAtributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [3]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape

(1000, 8)

In [5]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [8]:
y = dataset['CLASSE']
x = dataset.iloc[:,:-1]

In [9]:
x.isnull().sum()

,0
SALDO_ATUAL,7
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


In [10]:
mediana = x['SALDO_ATUAL'].median()
mediana

2323.0

Preenchendo valores nulos em Saldo Atual com a mediana.

In [14]:
x['SALDO_ATUAL'] = x['SALDO_ATUAL'].fillna(mediana)
x.isnull().sum()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


In [15]:
agrupado = x.groupby(['ESTADOCIVIL']).size()
agrupado

,0
ESTADOCIVIL,
fem div/cas,308
masculino casado/viuvo,92
masculino div/sep,50
masculino solteiro,542


Preenchendo valores nulos em 'ESTADOCIVIL' com a moda.

In [19]:
x['ESTADOCIVIL'] = x['ESTADOCIVIL'].fillna('masculino solteiro')
x.isnull().sum()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,0
PROPOSITO,0


Verificando
e removendo outliers

Primeiro verifico o Desvio Padrão:

In [20]:
desv = x['SALDO_ATUAL'].std()
desv

685936688.9820064

Atribui a mediana em locais aonde tinha desvio padrão no 'SALDO_ATUAL'

In [ ]:
x.loc[x['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL'] = mediana

In [26]:
agrupado = x.groupby(['PROPOSITO']).size()
agrupado

,0
PROPOSITO,
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,33
radio/tv,280


In [25]:
x.loc[x['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
x.loc[x['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'


In [34]:
x['DATA']

,DATA
0,2019-01-01
1,2020-01-01
2,2020-01-02
3,2019-01-02
4,2018-01-03
...,...
995,2018-06-29
996,2018-06-30
997,2018-07-03
998,2019-07-04


In [32]:
x['DATA'] = pd.to_datetime(x['DATA'], format='%d/%m/%Y')

In [38]:
x['ANO'] = x['DATA'].dt.year
x['MES'] = x['DATA'].dt.month
x['DIA'] = x['DATA'].dt.day
x['DIASEMANA'] = x['DATA'].dt.day_name()
x['DIASEMANA']

,DIASEMANA
0,Tuesday
1,Wednesday
2,Thursday
3,Wednesday
4,Wednesday
...,...
995,Friday
996,Saturday
997,Tuesday
998,Thursday


Adicionando valor numerico a atributos categoricos via label encoder:

In [42]:
x['ESTADOCIVIL'].unique()

array(['masculino solteiro'], dtype=object)

In [43]:
x['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [44]:
x['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [45]:
labelencoder = LabelEncoder()
x['ESTADOCIVIL'] = labelencoder.fit_transform(x['ESTADOCIVIL'])
x['PROPOSITO'] = labelencoder.fit_transform(x['PROPOSITO'])
x['DIASEMANA'] = labelencoder.fit_transform(x['DIASEMANA'])
x.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,0,7,2019,1,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,1,6
2,2096.0,3,49,nenhum,2020-01-02,0,2,2020,1,2,4
3,7882.0,4,45,nenhum,2019-01-02,0,3,2019,1,2,6
4,4870.0,4,53,nenhum,2018-01-03,0,0,2018,1,3,6


Aplicando ONE Hot-encoding

In [47]:
outros = x['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [48]:
z = pd.get_dummies(x['OUTROSPLANOSPGTO'], prefix = 'OUTROS')

In [50]:
 z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [51]:
x

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,0,7,2019,1,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,1,6
2,2096.0,3,49,nenhum,2020-01-02,0,2,2020,1,2,4
3,7882.0,4,45,nenhum,2019-01-02,0,3,2019,1,2,6
4,4870.0,4,53,nenhum,2018-01-03,0,0,2018,1,3,6
...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,29,0
996,3857.0,4,40,nenhum,2018-06-30,0,1,2018,6,30,2
997,804.0,4,38,nenhum,2018-07-03,0,7,2018,7,3,5
998,1845.0,4,23,nenhum,2019-07-04,0,7,2019,7,4,4


Padronizar atributos desbalanceados:

In [52]:
sc = StandardScaler()
m = sc.fit_transform(x.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

Juntar elementos z e m no df original e depois excluir atributos desnecessarios.

In [53]:
x = pd.concat([x,z,pd.DataFrame(m,columns=['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N'])], axis=1)

In [55]:
x.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,0,7,2019,1,1,5,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,1,6,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,0,2,2020,1,2,4,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,0,3,2019,1,2,6,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,0,0,2018,1,3,6,False,True,False,0.572709,1.046987,0.898982


In [56]:
x.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)

In [57]:
x

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,0,7,2019,1,1,5,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,1,6,True,False,0.957740,-0.765977,-0.740241
2,0,2,2020,1,2,4,True,False,-0.415337,0.140505,0.687469
3,0,3,2019,1,2,6,True,False,1.645526,1.046987,0.475956
4,0,0,2018,1,3,6,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,29,0,True,False,-0.543562,1.046987,-0.264338
996,0,1,2018,6,30,2,True,False,0.211898,1.046987,0.211566
997,0,7,2018,7,3,5,True,False,-0.875522,1.046987,0.105809
998,0,7,2019,7,4,4,True,False,-0.504738,1.046987,-0.687363
